In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import sys
from bs4 import BeautifulSoup
import time
import numpy as np
from datetime import datetime

In [2]:
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
def send_keys_delay_random(controller,keys,min_delay=0.05,max_delay=0.25):
    for key in keys:
        controller.send_keys(key)
        time.sleep(np.random.uniform(low=min_delay,high=max_delay))

In [4]:
def search_korean_air_once(driver, orig, dest, date):
    time.sleep(0.4)
    cnt = 0
    while cnt < 10:
        try:
            oneWayRadioBtn = driver.find_element_by_xpath('//*[@id="oneway"]')
            oneWayRadioBtn.click()
        except Exception as e:
            time.sleep(np.random.uniform(low=3, high=7))
            cnt += 1
        else:
            break
    time.sleep(1)

    fromInput = driver.find_element_by_xpath('//*[@id="from-to-chooser"]/div[2]/ul/li/div/div/div/input')
    fromInput.clear()
    time.sleep(0.7)
    send_keys_delay_random(fromInput, orig)

    toInput = driver.find_element_by_xpath('//*[@id="from-to-chooser"]/div[2]/ul/li[2]/div/div/div/input')
    toInput.clear()
    time.sleep(0.3)
    send_keys_delay_random(toInput, dest)

    dateInput = driver.find_element_by_xpath('//*[@id="from-to-chooser"]/div[2]/ul/li[3]/div/input')
    dateInput.clear()
    time.sleep(0.9)
    send_keys_delay_random(dateInput, date)

    time.sleep(1.2)
    findFlightsBtn = driver.find_element_by_xpath('//*[@id="submit"]')
    findFlightsBtn.click()

    time.sleep(2.11)
    # airMessageCheckBox = driver.find_element_by_xpath('//*[@id="airpmessage-checkbox"]')
    # airMessageCheckBox.click()
    # time.sleep(0.521)
    airMessageConfirmBtn = driver.find_element_by_xpath('//*[@id="btnModalPopupYes"]')
    airMessageConfirmBtn.click()
    time.sleep(1.6)
    cnt = 0
    while cnt < 10:
        notFoundMessageBtnExist = driver.find_elements_by_xpath('//*[@id="btnModalPopupYes"]')
        if not notFoundMessageBtnExist:
            cnt += 1
            time.sleep(np.random.uniform(low=2, high=4))
        else:
            break
            
    if not notFoundMessageBtnExist:
        t = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
        print("[{}] Found flight for {}-{} on {}.".format(t, orig, dest, date))
        return True
    else:
        notFoundMessageBtn = notFoundMessageBtnExist[0]
        notFoundMessageBtn.click()
        return False

In [5]:
def search_korean_air_second_round(driver, orig, dest, date):
    cnt = 0
    while cnt < 5:
        try:
            oneWayRadioBtn = driver.find_element_by_xpath('//*[@id="from-to-chooser"]/div[1]/ul/li/input')
        except Exception as e:
            time.sleep(np.random.uniform(low=2, high=4))
        else:
            break
    oneWayRadioBtn.click()
    
    fromInput = driver.find_element_by_xpath('//*[@id="boundList"]/ul/li[1]/div/div/div/input')
    fromInput.clear()
    time.sleep(0.7)
    send_keys_delay_random(fromInput, orig)
    time.sleep(1.1)
    
    toInput = driver.find_element_by_xpath('//*[@id="boundList"]/ul/li[2]/div/div/div/input')
    toInput.clear()
    time.sleep(0.91)
    send_keys_delay_random(toInput, dest)
    time.sleep(0.21)
    
    dateInput = driver.find_element_by_xpath('//*[@id="boundList"]/ul/li[3]/div/input')
    dateInput.clear()
    time.sleep(0.9)
    send_keys_delay_random(dateInput, date)
    
    time.sleep(0.72)
    try:
        currencyUSDRadioBtn = driver.find_element_by_xpath('//*[@id="currency_USD"]')
        currencyUSDRadioBtn.click()
    except Exception as e:
        pass
    time.sleep(1.5)
    submitBtn = driver.find_element_by_xpath('//*[@id="submit"]')
    submitBtn.click()
    
    time.sleep(2.11)
    # //*[@id="btnModalPopupYes"]
    airMessageConfirmBtn = driver.find_element_by_xpath('//*[@id="btnModalPopupYes"]')
    airMessageConfirmBtn.click()
    time.sleep(1.6)
    cnt = 0
    while cnt < 10:
        notFoundMessageBtnExist = driver.find_elements_by_xpath('//*[@id="btnModalPopupYes"]')
        if not notFoundMessageBtnExist:
            cnt += 1
            time.sleep(np.random.uniform(low=2, high=4))
        else:
            break
            
    if not notFoundMessageBtnExist:
        t = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
        print("[{}] Found flight for {}-{} on {}.".format(t, orig, dest, date))
        return True
    else:
        notFoundMessageBtn = notFoundMessageBtnExist[0]
        notFoundMessageBtn.click()
        return False

In [6]:
def get_flight_info_from_flight_item_html(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    flight_fare = soup.find(class_='flight-fare-passenger-type').text.strip()
    if 'unavailable' in flight_fare:
        return None
    print('flight_fare: {}'.format(flight_fare))
    flight_summary = soup.find(class_='flight-summary')
#     flight_date = flight_summary.find(class_='flight-date').text.strip()
#     print('flight_date: {}'.format(flight_date))
#     flight_time = flight_summary.find(class_='flight-time').text.strip()
#     print('flight_time: {}'.format(flight_time))
    flight_total_time = flight_summary.find(class_='flight-total-time').text.strip()
    print('flight_total_time: {}'.format(flight_total_time))
    flight_stop = flight_summary.find(class_='flight-stop').text.strip()
    print('flight_stop: {}'.format(flight_stop))
    flight_number = soup.find(class_='flight')['data-flight-number'].strip()
    print('flight_number: {}'.format(flight_number))
    flight_class = soup.find(class_='planeType').text.strip()
    print('flight_class: {}'.format(flight_class))
    flight_route_from, flight_from_time = soup.find(class_='route from').text.strip().split()
    flight_route_to, flight_to_time = soup.find(class_='route to').text.strip().split()
    print('{} {} -> {} {}'.format(flight_route_from, flight_from_time, flight_route_to, flight_to_time))
    result = {
        'flight_fare': flight_fare,
        'flight_duration': flight_total_time,
        'flight_stop': flight_stop,
        'flight_number': flight_number,
        'flight_class': flight_class,
        'flight_route_from': flight_route_from,
        'flight_route_to': flight_route_to,
        'flight_from_time': flight_from_time,
        'flight_to_time': flight_to_time,
        'timestamp': datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
    }
    return result

In [7]:
def get_korean_air_flight_info(driver, orig, dest, date):
    time.sleep(2.11)
    continueBtn = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[2]/button')
    continueBtn.click()
    cnt = 0
    time.sleep(np.random.uniform(low=2, high=6))
    while cnt < 5:
        try:
            header = driver.find_element_by_xpath('//*[@id="other-pannel"]/div[1]')
        except Exception as e:
            time.sleep(np.random.uniform(low=2, high=6))
            cnt += 1
        else:
            break
    flight_items = driver.find_elements_by_class_name('flightItem')
    flight_info_list = []
    for flight_item in flight_items:
        temp_html = flight_item.get_attribute('innerHTML')
        flight_info = get_flight_info_from_flight_item_html(temp_html)
        if flight_info is not None:
            flight_info_list.append(flight_info)
    return flight_info_list

In [8]:
def search_korean_air_date_range(orig, dest, dates):
    driver = webdriver.Chrome()
    driver.delete_all_cookies()
    driver.set_window_size(1024, 600)
    url = "https://www.koreanair.com/global/en.html"
    driver.get(url)
    time.sleep(20)
    results = {}
    for i, date in enumerate(dates):
        params = {
            'orig': orig,
            'dest': dest,
            'date': date
        }
        if i == 0:
            result = search_korean_air_once(driver, **params)
        else:
            result = search_korean_air_second_round(driver, **params)
        if result:
            info = get_korean_air_flight_info(driver, **params)
            if len(info) > 0:
                results[date] = info
    driver.quit()
    return results

In [9]:
params = {
    'orig': 'Seoul/Incheon(ICN)',
    'dest': 'Shenyang(SHE)',
    'dates': ['2020-08-21', '2020-11-27']
}

In [10]:
results = search_korean_air_date_range(**params)
print('\n\n===============\nResults:\n----------\n')
print(results)

[05-Jul-2020 (21:01:56.353740)] Found flight for Seoul/Incheon(ICN)-Shenyang(SHE) on 2020-11-27.
flight_fare: KRW 249,000   Adult
flight_total_time: 1h 55m
flight_stop: Non-Stop
flight_number: KE831
flight_class: B777-300  Open new window for the fleet information
Seoul/Incheon(ICN) 08:50 -> Shenyang(SHE) 09:45


Results:
----------

{'2020-11-27': [{'flight_fare': 'KRW 249,000\xa0 \xa0Adult', 'flight_duration': '1h 55m', 'flight_stop': 'Non-Stop', 'flight_number': 'KE831', 'flight_class': 'B777-300\xa0 Open new window for the fleet information', 'flight_route_from': 'Seoul/Incheon(ICN)', 'flight_route_to': 'Shenyang(SHE)', 'flight_from_time': '08:50', 'flight_to_time': '09:45'}]}


In [ ]:
# get_flight_info_from_flight_item_html(results['2020-11-27'][1])

In [ ]:
# soup = BeautifulSoup(results['2020-11-27'][1], 'html.parser')
# print(soup.prettify())